In [ ]:
# this is for logging, ignore this cell if you are not using arize
import llama_index.core
llama_index.core.set_global_handler("arize_phoenix")

The following example demonstrates a basic supervisor workflow.

1.  **Define Tools**: Functions for `add`, `multiply`, and `web_search` are defined and converted into `FunctionTool` objects.
2.  **Create Agents**: Two specialized `FunctionAgent` instances are created:
    *   `math_agent`: Uses `add_tool` and `multiply_tool`.
    *   `research_agent`: Uses `search_tool`.
3.  **Initialize Supervisor**: A `Supervisor` is created to manage the `math_agent` and `research_agent`. The `add_tree_structure=True` adds json context to the llm about the hierarchy of agents and tools.
4.  **Run Workflow**: The supervisor is executed with the input query "what's the combined headcount of the FAANG companies in 2024?". The supervisor delegates the task to the appropriate agent(s) based on the query and tool capabilities. First, the `research_agent` is likely called to find the headcounts, and then the `math_agent` is called to sum them up.
5.  **Print Response**: The final response from the supervisor is printed.

In [2]:
from llama_index.llms.openai import OpenAI
from llama_index_supervisor import Supervisor
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.workflow import Context

# Initialize the LLM
llm = OpenAI(model="gpt-4o", temperature=0)

# Define tools
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

def web_search(query: str) -> str:
    """Search the web for information."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )

# Create function tools
add_tool = FunctionTool.from_defaults(fn=add)
multiply_tool = FunctionTool.from_defaults(fn=multiply)
search_tool = FunctionTool.from_defaults(fn=web_search)

# Create specialized agents
math_agent = FunctionAgent(
    name="math_expert",
    llm=llm,
    tools=[add_tool, multiply_tool],
    system_prompt="You are a math expert. Always use one tool at a time."
)

research_agent = FunctionAgent(
    name="research_expert",
    llm=llm,
    tools=[search_tool],
    system_prompt="You are a world class researcher with access to web search. Do not do any math."
)

# Create supervisor workflow
supervisor = Supervisor(
    llm=llm,
    agents=[math_agent, research_agent],
    add_tree_structure=True,
    timeout=60
)

# Run the workflow
ctx = Context(supervisor)
response = await supervisor.run(
    input="what's the combined headcount of the FAANG companies in 2024?",
    ctx=ctx
)

print(str(response))

assistant: The combined headcount of the FAANG companies in 2024 is 1,977,586 employees.


This example showcases a hierarchical supervisor structure:

1.  **Define Tools**: Functions for `add`, `multiply`, `web_search`, `format_document`, and `publish_content` are defined and converted into `FunctionTool` objects.
2.  **Create Base Agents**: Four specialized `FunctionAgent` instances are created: `math_agent`, `research_agent`, `writing_agent`, and `publishing_agent`, each equipped with relevant tools.
3.  **Create Mid-Level Supervisors**: Two `Supervisor` instances are created:
    *   `research_supervisor`: Manages `research_agent` and `math_agent`.
    *   `writing_supervisor`: Manages `writing_agent` and `publishing_agent`.
4.  **Create Top-Level Supervisor**: A `top_level_supervisor` is created to manage the `research_supervisor` and `writing_supervisor`, establishing a three-tier hierarchy. The `add_tree_structure=True` adds json context to the llm about the hierarchy of agents and tools.
5.  **Run Workflow**: The `top_level_supervisor` is executed with the input query "what's the combined headcount of the FAANG companies in 2024?". The top supervisor delegates the task to the appropriate mid-level supervisor (`research_supervisor`), which in turn delegates to the appropriate base agent(s) (`research_agent` then `math_agent`).
6.  **Print Response**: The final response coordinated through the hierarchy is printed.

In [9]:
from llama_index.llms.openai import OpenAI
from llama_index_supervisor import Supervisor
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.workflow import Context

# Initialize the LLM
llm = OpenAI(model="gpt-4o", temperature=0)

# Define all the required tools
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

def web_search(query: str) -> str:
    """Search the web for information."""
    return "Web search results for: " + query

def format_document(content: str) -> str:
    """Format document with proper structure and styling."""
    return f"Formatted document: {content}"

def publish_content(content: str) -> str:
    """Publish content to appropriate channels."""
    return f"Published: {content}"

# Create function tools
add_tool = FunctionTool.from_defaults(fn=add)
multiply_tool = FunctionTool.from_defaults(fn=multiply)
search_tool = FunctionTool.from_defaults(fn=web_search)
format_tool = FunctionTool.from_defaults(fn=format_document)
publish_tool = FunctionTool.from_defaults(fn=publish_content)

# Create base-level agents
math_agent = FunctionAgent(
    name="math_expert",
    llm=llm,
    tools=[add_tool, multiply_tool],
    system_prompt="You are a math expert. Always use one tool at a time."
)

research_agent = FunctionAgent(
    name="research_expert", 
    llm=llm,
    tools=[search_tool],
    system_prompt="You are a world class researcher with access to web search. Do not do any math."
)

writing_agent = FunctionAgent(
    name="writing_expert",
    llm=llm,
    tools=[format_tool],
    system_prompt="You are a professional writer who formats and improves content."
)

publishing_agent = FunctionAgent(
    name="publishing_expert",
    llm=llm,
    tools=[publish_tool],
    system_prompt="You are a publishing expert who knows how to distribute content effectively."
)

# Create mid-level supervisors (research team and writing team)
research_supervisor = Supervisor(
    llm=llm,
    agents=[research_agent, math_agent],
    name="research_supervisor",
    system_prompt="You manage a research team with experts in research and math. Delegate tasks appropriately.",   
    timeout=60
)

writing_supervisor = Supervisor(
    llm=llm,
    agents=[writing_agent, publishing_agent],
    name="writing_supervisor", 
    system_prompt="You manage a content team with experts in writing and publishing. Delegate tasks appropriately.",
    timeout=60
)

# Create top-level supervisor
top_level_supervisor = Supervisor(
    llm=llm,
    agents=[research_supervisor, writing_supervisor],
    name="top_level_supervisor",
    system_prompt="You are the executive supervisor coordinating between the research and writing teams. For research or math problems, use the research_supervisor. For content creation and publishing, use the writing_supervisor.",
    timeout=60,
    add_tree_structure=True
)

query = "what's the combined headcount of the FAANG companies in 2024?"

ctx = Context(top_level_supervisor)
response = await top_level_supervisor.run(
    input=query,
    ctx=ctx,
)

print(response)

assistant: The research expert was unable to find explicit headcount numbers for the FAANG companies in 2024 through web searches. It is recommended to check the latest annual reports or press releases from each company for the most accurate and up-to-date information. If you need further assistance or a different approach, please let me know!


In [4]:
## adding chat messages
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4o", temperature=0)
from llama_index.core.llms.llm import ChatMessage
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.workflow import Context
from llama_index_supervisor import Supervisor

ctx = Context(
    supervisor
)
memory=ChatMemoryBuffer.from_defaults(
    chat_history=[
        # these messages will be added to the chat history right after the system prompt
        ChatMessage(role="user", content="what's the combined headcount of the FAANG companies in 2024?"),
        ChatMessage(role="assistant", content="The combined headcount is 1,000,000."),
    ]
)
await ctx.set("memory", memory)

supervisor = Supervisor(llm=llm, agents=[math_agent, research_agent], system_prompt="You are a math expert and a world class researcher with access to web search.")
await supervisor.run(
    input="Paraphrase your answer.",
    ctx=ctx,
)

# chat history will look like this
# [
#     ChatMessage(role="system", content="You are a math expert and a world class researcher with access to web search."),
#     ChatMessage(role="user", content="what's the combined headcount of the FAANG companies in 2024?"),
#     ChatMessage(role="assistant", content="The combined headcount is 1,000,000."),
#     ChatMessage(role="user", content="Paraphrase your answer.")
#]

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The total number of employees across all FAANG companies in 2024 is 1,000,000.')]), raw=ChatCompletionChunk(id='chatcmpl-BJKyiHSvUAIe43ZbrzFn0twZebwKk', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, refusal=None, role=None, tool_calls=None), finish_reason='stop', index=0, logprobs=None)], created=1743948572, model='gpt-4o-2024-08-06', object='chat.completion.chunk', service_tier='default', system_fingerprint='fp_898ac29719', usage=None), delta='', logprobs=None, additional_kwargs={})

In [5]:

from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4o", temperature=0)
from llama_index.core.llms.llm import ChatMessage
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.workflow import Context
from llama_index_supervisor import Supervisor

### you can also use your own workflow as an agent and pass it into the supervisor
from llama_index.core.workflow import Workflow, Context, StartEvent, StopEvent, step
from llama_index.core.llms import LLM


class CoolAgent(Workflow):
    def __init__(self, name: str, description: str, llm: LLM, system_prompt: str = "You are a cool agent. Be cool."):
        super().__init__()
        self.name = name
        self.description = description
        self.llm = llm
        self.system_message = ChatMessage(role="system", content=system_prompt)
    @step
    async def start_flow(self, ctx: Context, ev: StartEvent) -> StopEvent:
        # the supervisor send a copy of the chat history through the ctx
        memory: ChatMemoryBuffer = await ctx.get("memory", default=ChatMemoryBuffer.from_defaults(llm=self.llm))
        # you can use the memory to get the chat history
        # messages = memory.get() # or get_all()
        # do something with the messages
        await memory.aput(
            ChatMessage(role="user", content="How are you?")
        )
        response = await self.llm.achat([self.system_message] + memory.get()) # dont add the system message to the memory so it doesnt get sent back to the supervisor
        
        # add response to the memory
        await memory.aput(response.message)
        # set ctx("memory") to the updated memory (just in case no memory key in context was passed in the first place)
        await ctx.set("memory", memory)
        return StopEvent(result=response.message) # the supervisor doesn't check the result of the workflow, so you can return anything you want in case you use the agent
    

cool_agent = CoolAgent(
    name="cool_agent",
    description="A cool agent that does cool things.",
    llm=llm,
    system_prompt="You are a cool agent. Be super super cool."
)

supervisor = Supervisor(
    llm=llm,
    agents=[cool_agent]
)

result = await supervisor.run(
    input="Ask the cool agent how it is doing."
)


In [10]:
print(result)

assistant: The cool agent says it's "doing as cool as a cucumber in a bowl of hot sauce!" How about you?
